In [6]:
from dataclasses import dataclass
from typing import Union, Type, Any
import re

- Classes: Setting, Variable, Unit
- Methods of Variable: add, sub, multi, negative (magic variables)
- Methods of Unit: mult, inverse $\left (\frac{1}{Unit} \right )$

In [7]:
STANDARD_UNIT_PREFIXES = {
    'n': 1e-9,
    'u': 1e-6,
    'm': 1e-3,
    'c': 1e-2,
    'd': 1e-1,
    '': 1,
    'k': 1e3,
    'M': 1e6,
    'G': 1e9,
    'T': 1e12,
}

In [8]:
@dataclass 
class Unit:
    """
    Класс единиц измерений
    """
    m: int
    s: int
    g: int
    Hz: int
    B: int
        
    def __mul__(self, other):
        """
        Операция перемножения степеней 
        """
        u = Unit(m= self.m + other.m,
                 s= self.s + other.s,
                 g= self.g + other.g,
                 Hz= self.Hz + other.Hz,
                 B= self.B + other.B)
        
        return u
    
    def inverse(self):
        """
        Операция обращение степеней
        Пример: kg^3 s^2 -> kg^-3 s^-2
        """
        u = Unit(m= -self.m,
                 s= -self.s,
                 g= -self.g,
                 Hz= -self.Hz,
                 B= -self.B)
        
        return u
            

In [9]:
@dataclass 
class Variable:
    """
    Класс переменной 
    """
    name: ''
    value: Union[int, float, bool, str]
    unit: Unit
    description: ''
        
    def __add__(self, other):
        """
        Сложение класса Variable
        
        Знак операции +
        """
        
        assert(self.unit.m == other.unit.m)
        assert(self.unit.s == other.unit.s)
        assert(self.unit.kg == other.unit.kg)
        assert(self.unit.Hz == other.unit.Hz)
        assert(self.unit.B == other.unit.B)
        
        var = Variable(name='',
                       value=self.value + other.value,
                       unit=self.unit,
                       description=f'Sum of {self.name} and {other.name}' )
        
        return var
    
    def __sub__(self, other):
        """
        Вычитание класса Variable
        
        Знак операции -
        """
        
        assert(self.unit.m == other.unit.m)
        assert(self.unit.s == other.unit.s)
        assert(self.unit.kg == other.unit.kg)
        assert(self.unit.Hz == other.unit.Hz)
        assert(self.unit.B == other.unit.B)
        
        var = Variable(name='',
                       value=self.value - other.value,
                       unit=self.unit,
                       description=f'Difference of {self.name} and {other.name}' )
        return var
    
    def __mul__(self, other):
        """
        Умножение класса Variable
        
        Знак операции *
        """
        
        u = self.unit * other.unit
        var = Variable(name='',
                       value=self.value * other.value,
                       unit=u,
                       description=f'The product of {self.name} and {other.name}')
        
        return var
    
    def __truediv__(self, other):
        """
        Деление класса Variable
        
        Знак операции /
        """
                
        u = self.unit * Unit.inverse(other.unit)
        
        var = Variable(name='',
                       value= self.value/other.value,
                       unit=u,
                       description=f'Fraction of {self.name} and {other.name}')
        
        return var
        
    def __neg__(self):
        """
        Операция -
        
        """
        
        var = Variable(name='',
                       value= -self.value,
                       unit=self.unit,
                       description=f'Neg of {self.name}')
        return var

In [10]:
a = Unit(1,0,3 ,4,5)
b =  Unit(1,2,3 ,4,5)
c = Variable('ha', 3, a, 'smth')
d = Variable('ha', -3, b, 'smth')

In [11]:
c

Variable(name='ha', value=3, unit=Unit(m=1, s=0, g=3, Hz=4, B=5), description='smth')

In [12]:
d

Variable(name='ha', value=-3, unit=Unit(m=1, s=2, g=3, Hz=4, B=5), description='smth')

In [13]:
a = c/d

In [14]:
@dataclass 
class Setting:
    """
    Класс настройки
    """
    name: ''
    default_value: Variable
    description: ''

In [15]:
# meter = {'nm': -9, 'um': -6, 'mm': -3, 'cm': -2, 'dm': -1 ,'m': 1, 'km': 3,'Mm': 6,'Gm': 9,'Tm': 12}
# sec = {'ns': -9, 'us': -6, 'ms': -3, 'cs': -2, 'ds': -1 ,'s': 1, 'ks': 3,'Ms': 6,'Gs': 9,'Ts': 12}
# gram = {'ng': -9, 'ug': -6, 'mg': -3, 'cg': -2, 'dg': -1 ,'g': 1, 'kg': 3,'Mg': 6,'Gg': 9,'Tg': 12}
# hertz = {'nHz':-9, 'uHz':-6, 'mHz':-3, 'cHz':-2, 'dHz':-1 ,'Hz':1, 'kHz':3,'MHz':6,'GHz':9,'THz':12}
# tesla = {'nB': -9, 'uB': -6, 'mB': -3, 'cB': -2, 'dB': -1 ,'B': 1, 'kB': 3,'MB': 6,'GB': 9,'TB': 12}

# dict_units = {'m':meter, 's': sec, 'g':gram, 'Hz':hertz, 'B': tesla}

dict_units = {
        'nm': -9, 'um': -6, 'mm': -3, 'cm': -2, 'dm': -1 ,'m': 1, 'km': 3,'Mm': 6,'Gm': 9,'Tm': 12,
        'ns': -9, 'us': -6, 'ms': -3, 'cs': -2, 'ds': -1 ,'s': 1, 'ks': 3,'Ms': 6,'Gs': 9,'Ts': 12,
        'ng': -9, 'ug': -6, 'mg': -3, 'cg': -2, 'dg': -1 ,'g': 1, 'kg': 3,'Mg': 6,'Gg': 9,'Tg': 12,
        'nHz':-9, 'uHz':-6, 'mHz':-3, 'cHz':-2, 'dHz':-1 ,'Hz':1, 'kHz':3,'MHz':6,'GHz':9,'THz':12,
        'nB': -9, 'uB': -6, 'mB': -3, 'cB': -2, 'dB': -1 ,'B': 1, 'kB': 3,'MB': 6,'GB': 9,'TB': 12}

In [16]:
def input_setting(s):
    
    d = {'m':0, 's':0, 'g':0, 'Hz':0, 'B':0}
    
    splited_string = s.split()
    value = float(splited_string[0])
    
    for i in splited_string[1:]:
        if (i.find('^') == -1):
            if (i.find('s') != -1):
                d['s'] += dict_units[i]
            elif (i.find('g') != -1):
                d['g'] += dict_units[i] 
            elif (i.find('Hz') != -1):
                d['Hz'] += dict_units[i] 
            elif (i.find('B') != -1):
                d['B'] += dict_units[i] 
            else:
                d['m'] += dict_units[i]
                
        else:
            i = i.split('^')
            if (i[0].find('s') != -1):
                d['s'] += dict_units[i[0]] * int(i[-1])
            elif (i[0].find('g') != -1):
                d['g'] += dict_units[i[0]] * int(i[-1])
            elif (i[0].find('Hz') != -1):
                d['Hz'] += dict_units[i[0]] * int(i[-1])
            elif (i[0].find('B') != -1):
                d['B'] += dict_units[i[0]] * int(i[-1])
            else:
                d['m'] += dict_units[i[0]] * int(i[-1])
                
    
    u = Unit(d['m'], d['s'], d['g'], d['Hz'], d['B'] )
    
    var = Variable(name='',
          value=value, 
          unit=u,
          description='')
    
    
    return var

In [25]:
s = '10000 m^1 Gs       kg^3 Hz^4 B^-5'


In [26]:
input_setting(s)

Variable(name='', value=10000.0, unit=Unit(m=1, s=9, g=9, Hz=4, B=-5), description='')

In [19]:
OPERATORS = {'+': (1, lambda x, y: x + y), '-': (1, lambda x, y: x - y),
             '*': (2, lambda x, y: x * y), '/': (2, lambda x, y: x / y)}


def eval_(formula):
    def parse(formula_string):
        number = ''
        for s in formula_string:
            if s in '1234567890.':
                number += s
            elif number:
                yield float(number)
                number = ''
            if s in OPERATORS or s in "()":
                yield s
        if number:
            yield float(number)

    def shunting_yard(parsed_formula):
        stack = []
        for token in parsed_formula:
            if token in OPERATORS:
                while stack and stack[-1] != "(" and OPERATORS[token][0] <= OPERATORS[stack[-1]][0]:
                    yield stack.pop()
                stack.append(token)
            elif token == ")":
                while stack:
                    x = stack.pop()
                    if x == "(":
                        break
                    yield x
            elif token == "(":
                stack.append(token)
            else:
                yield token
        while stack:
            yield stack.pop()

    def calc(polish):
        stack = []
        for token in polish:
            if token in OPERATORS:
                y, x = stack.pop(), stack.pop()
                stack.append(OPERATORS[token][1](x, y))
            else:
                stack.append(token)
        return stack[0]

    return calc(shunting_yard(parse(formula)))

In [24]:
eval_('2+3*((1 + 2.3)+1)/1')

14.899999999999999

- добавить степень
- постараться получить калькулятор с размерностью 
- добавить метод сравнения в класс юнит
- сделать двумя форами автоматическую генерацию дик унит